In [6]:
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np

n_input1 = 28 # pixels of a digit in MNIST
n_input2 = n_input1 * 2 
n_kernel = 3 # CNN with a kernel of 3 by 3 
n_pool = 2 # CNN with a pooling of 2 by 2
n_output = 10 # corresponds to the 10 digits (0-9)
n_batch_size = 50 
n_epochs = 5

# required orderwise tf uses up all the memory of GPU and crashes 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

# getting the MNIST dataset : 60k of handwritten labelled digits 

mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], n_input1, n_input1, 1)
x_test = x_test.reshape(x_test.shape[0], n_input1, n_input1, 1)

input_shape = (n_input1, n_input1, 1)

# convert from integer to float

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# normalize to range 0-1

x_train /= 255.0
x_test /= 255.0

# Building up the model 

model = keras.models.Sequential() 

model.add(keras.layers.Conv2D(n_input1,(n_kernel,n_kernel), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
model.add(keras.layers.MaxPooling2D(n_pool,n_pool))
model.add(keras.layers.Conv2D(n_input2,(n_kernel,n_kernel),activation= 'relu', kernel_initializer='he_uniform'))
model.add(keras.layers.Conv2D(n_input2,(n_kernel,n_kernel),activation='relu', kernel_initializer='he_uniform'))       
model.add(keras.layers.MaxPooling2D(pool_size=(n_pool,n_pool)))  
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100,activation='relu', kernel_initializer='he_uniform'))
model.add(keras.layers.Dropout(0.2)) # avoid overfitting 
model.add(keras.layers.Dense(n_output, activation='softmax'))

# Setting the optimizer and the loss function hyperparameters

opt = keras.optimizers.SGD(lr=0.01, momentum=0.9)
model.compile (optimizer=opt,
                loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# training of the model on the GPU

model.fit(x=x_train,y=y_train, batch_size=n_batch_size, epochs=n_epochs)

# evaluation of the model 

model.evaluate(x_test, y_test)

# save the model 

model.save('handwritten_digit_model.h5')

1 Physical GPUs, 1 Logical GPUs
Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 54us/sample - loss: 0.1887 - accuracy: 0.9399
Epoch 2/5
60000/60000 [==============================] - 3s 49us/sample - loss: 0.0626 - accuracy: 0.9812
Epoch 3/5
60000/60000 [==============================] - 3s 49us/sample - loss: 0.0451 - accuracy: 0.9858
Epoch 4/5
60000/60000 [==============================] - 3s 49us/sample - loss: 0.0359 - accuracy: 0.9889
Epoch 5/5
10000/10000 [==============================] - 1s 55us/sample - loss: 0.0289 - accuracy: 0.9904


In [7]:
# load the model

model = keras.models.load_model('handwritten_digit_model.h5')

# load the example image 

filename = "digit.png" # put your image including file extension
image = keras.preprocessing.image.load_img(filename,color_mode='grayscale', target_size=(n_input1,n_input1))

display("image: ", image)

#convert to array 

image = keras.preprocessing.image.img_to_array(image)

#reshape into a single sample with 1 channel

image = image.reshape(1,n_input1,n_input1,1)

#prepare pixel data

image = image.astype('float32')
image = image / 255.0

#predict the digit 

digit = model.predict_classes(image)
display("prediction ", digit[0])

'image: '

'prediction '

5